<a href="https://colab.research.google.com/github/Feindrial/bondnet/blob/main/main1m.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
dxbatch = []
        dybatch = []
        while True:
            for u in self.shuffled_users:            
                item_query = self.data[self.data["user_id"] == u].sort_values("timestamp")["item_id"].to_list()

                index = 0
                while (len(item_query) - index - 1) >= self.window_length:
                    cur_query = item_query[index:index + self.window_length]
                    u_index = np.argwhere(self.user_index_query==u)[0][0]
                    
                    dxtemp = []
                    for i in cur_query:
                        i_index = np.argwhere(self.item_index_query==i)[0][0]
                        
                        dxhalf = [] 
                        dxhalf.append([self.svdpp.pu[u_index], self.svdpp.qi[i_index]])
                        dxhalf = np.asanyarray(dxhalf).flatten().tolist()
                        dxhalf.append(self.svdpp.bu[u_index])
                        dxhalf.append(self.svdpp.bi[i_index])
                        dxtemp.append(np.asanyarray(dxhalf))

                    dxbatch.append(np.asanyarray(dxtemp))    
                    dybatch.append(np.asanyarray(self.data[(self.data["user_id"] == u) & (self.data["item_id"] == item_query[index])]["rating"]))     

                    if len(dxbatch) == self.batch_size:                    
                        yield np.asanyarray(dxbatch), np.asanyarray(dybatch)
                        dxbatch = []
                        dybatch = []

                    index += 1

In [1]:
import numpy as np
import pandas as pd
import sklearn as sk
import tensorflow as tf
from tensorflow import keras
from pathlib import Path

print(tf.__version__)

locc = "COLAB"

if locc == "LOCAL":

    #veri setinin indirilip dosya yollarının kaydedilmesi

    movielens_url = "http://files.grouplens.org/datasets/movielens/ml-latest-small.zip"

    downloaded_path = tf.keras.utils.get_file(movielens_url.rsplit('/').pop(), movielens_url, extract=True)

    dataset_dir = Path(downloaded_path).parent / Path(downloaded_path).stem
    print(dataset_dir)

    #indirelen veri setinin ilgili csv dosyaları pandas dataframe olarak atandı

elif locc == "COLAB":

    dataset_dir = Path("/content")
    print(dataset_dir)

    ! pip install -q kaggle
    from google.colab import files
    files.upload()

    ! mkdir ~/.kaggle
    ! mv kaggle.json ~/.kaggle/
    ! chmod 600 ~/.kaggle/kaggle.json

    !kaggle datasets download "odedgolden/movielens-1m-dataset"
    !unzip -j "movielens-1m-dataset"

else:
    raise RuntimeError


ratings_file = dataset_dir / "ratings.dat"
ratings = pd.read_csv(
    ratings_file,
    sep = "::",
    names = ['userId', 'movieId', 'rating', 'timestamp']
)

movies_file = dataset_dir / "movies.dat"
movies = pd.read_csv(
    movies_file,
    sep = "::",
    names = ['movieId', 'title', 'genres'],
    encoding='latin-1',
    engine='python',
)

#TODO doğal dil işleme ile kullanıcıların ilgili filme yaptığı yorumlar çıkarılabilir

# tags_file = dataset_dir / "tags.dat"
# tags = pd.read_csv(tags_file)

2.12.0
/content


mkdir: cannot create directory ‘/root/.kaggle’: File exists
mv: cannot stat 'kaggle.json': No such file or directory
movielens-1m-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  movielens-1m-dataset.zip
replace README? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


<ipython-input-1-b5dccf25f3d9>:46: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv(


In [2]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype
---  ------     --------------    -----
 0   userId     1000209 non-null  int64
 1   movieId    1000209 non-null  int64
 2   rating     1000209 non-null  int64
 3   timestamp  1000209 non-null  int64
dtypes: int64(4)
memory usage: 30.5 MB


In [3]:
ratings.head(10)

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [4]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  3883 non-null   int64 
 1   title    3883 non-null   object
 2   genres   3883 non-null   object
dtypes: int64(1), object(2)
memory usage: 91.1+ KB


In [5]:
movies.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [6]:
#filmlerin kaç farklı türü olduğu bulundu ve bu bulunan türler "movies" dataframe'ine sütun olarak eklendi
unique_genres = pd.unique(movies["genres"].str.split("|", expand=True).values.ravel())
movies[unique_genres] = 0

#hafızada daha az yer kaplaması açısından bu film türlerini tutan sütunlar 1 byte olarak tutuldu
movies = movies.astype({x: np.int8 for x in unique_genres})

#bir yukarıdaki hücrede de görüleceği üzere indirilen dataframe'de film türleri tek sütünda ve "|" sembolü ile ayrılmış durumda
#üstteki 3 satırlık kodda bu türleri kendi ayrı sütunlarına almıştık 
#alttaki for döngüsünde ise ilgili filme denk gelen tür sütunları eğer bu filmin türünü barındırıyor ise 1 yapıldı
#eğer barındırmıyor ise üstteki ilgili koddan dolayı 0 olarak kaldı
#örneğin "3" numaralı filmin türleri "Comedy|Romance" olduğu için
#movies dataframe'inin "Comedy" ve "Romance" sütununun değerleri "3" numaralı film için 1 oldu
for index, row in enumerate(movies["genres"].str.split("|")):
    movies.loc[index, row] = 1

movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   movieId      3883 non-null   int64 
 1   title        3883 non-null   object
 2   genres       3883 non-null   object
 3   Animation    3883 non-null   int8  
 4   Children's   3883 non-null   int8  
 5   Comedy       3883 non-null   int8  
 6   None         3883 non-null   int8  
 7   Adventure    3883 non-null   int8  
 8   Fantasy      3883 non-null   int8  
 9   Romance      3883 non-null   int8  
 10  Drama        3883 non-null   int8  
 11  Action       3883 non-null   int8  
 12  Crime        3883 non-null   int8  
 13  Thriller     3883 non-null   int8  
 14  Horror       3883 non-null   int8  
 15  Sci-Fi       3883 non-null   int8  
 16  Documentary  3883 non-null   int8  
 17  War          3883 non-null   int8  
 18  Musical      3883 non-null   int8  
 19  Mystery      3883 non-null 

In [7]:
#bir yukarıdaki hücrede bu türleri filmlere göre haritaladığımız için
#dataframe'de bulunan "genres" ve "title" sütunlarını hafızadan kazanmak için sildik
#böylece elimizde sadece ilgili filmin "id"si ve türleri kaldı
movies = movies.drop(["genres", "title"], axis=1)
movies.head(10)

,movieId,Animation,Children's,Comedy,None,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,5,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,6,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0
6,7,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
7,8,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,9,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
9,10,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0


In [8]:
#rating dataframe'inde ilgili kullanıcının ilgili filme kaç puan verdiği bulunuyor ama bu filmin hangi türlere sahip olduğu bulunmuyor
#bu yüzden yukarıdaki hücrelerde oluşturduğumuz "movies" dataframe'indeki ilgili "id"
#ratings dataframe'indeki "movieId"ler ile haritalandı
ratings = ratings.join(movies.set_index("movieId"), on="movieId", how="inner")

In [9]:
#kullanılmadığı için "timestamp" sütunu dataframe'den silindi ve
#kullanılacak modelin veriyi ezberlemesinden kaçınmak için dataframe'in satırları rastgele olarak karıştırıldı
ratings = ratings.drop("timestamp", axis=1)
ratings = ratings.sample(frac=1, random_state=1337)
ratings.head(10)

,userId,movieId,rating,Animation,Children's,Comedy,None,Adventure,Fantasy,Romance,...,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
590723,3603,2879,5,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
47835,320,3751,4,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
63187,424,1762,3,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
805379,4821,1196,3,0,0,0,0,1,0,0,...,0,0,0,1,0,1,0,0,0,0
197967,1216,1963,5,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
305764,1820,2455,5,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
397420,2353,2757,5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
396260,2343,2946,3,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
646225,3889,319,4,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
618703,3749,708,3,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [10]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000209 entries, 590723 to 792048
Data columns (total 22 columns):
 #   Column       Non-Null Count    Dtype
---  ------       --------------    -----
 0   userId       1000209 non-null  int64
 1   movieId      1000209 non-null  int64
 2   rating       1000209 non-null  int64
 3   Animation    1000209 non-null  int8 
 4   Children's   1000209 non-null  int8 
 5   Comedy       1000209 non-null  int8 
 6   None         1000209 non-null  int8 
 7   Adventure    1000209 non-null  int8 
 8   Fantasy      1000209 non-null  int8 
 9   Romance      1000209 non-null  int8 
 10  Drama        1000209 non-null  int8 
 11  Action       1000209 non-null  int8 
 12  Crime        1000209 non-null  int8 
 13  Thriller     1000209 non-null  int8 
 14  Horror       1000209 non-null  int8 
 15  Sci-Fi       1000209 non-null  int8 
 16  Documentary  1000209 non-null  int8 
 17  War          1000209 non-null  int8 
 18  Musical      1000209 non-null  int8 
 

In [11]:
#daha sonra kullanmak adına tekil kullanıcı sayısı ve film sayısı bulundu
num_unique_users = len(ratings["userId"].unique())
num_unique_movies = len(ratings["movieId"].unique())

#daha sonra kullanmak adına herhangi bir kullanıcı tarafından verilen en düşük ve en yüksek puan bulundu
min_rating = ratings["rating"].min()
max_rating = ratings["rating"].max()

#modele giriş olarak verilecek değerler x_train değişkeninde toplandı 
#bunlar kullanıcı, film "id"leri ve ilgili filmin türleri
x_train = ratings.drop("rating", axis=1).values

#filmlere kullanıcılar tarafından verilen puanlar reel sayılar kümesinde tanımlı olmadığı için
#örneğin 3.123 veya 1.567 olmadığı; 0.5, 2.5, 4.0 gibi belirli bir oranda artan sonlu sayılar kümesini oluşturduğu için
#bu değerler kategorisel olarak tutuldu
#örneğin bir kullanıcı bir filme 2.5 verdiyse min_rating değişkeninde başlamak ve max_rating değişkeninde bitmek üzere
#ilgili oranda artan(bu durumda 0.5) bir kategori kümesi oluşturuldu
#bu kategori kümesinde 2.5 puanı hangi kategori indisine denk geliyorsa ise o kategori indisinin değeri 1 yapıldı
y_labels = ratings["rating"]

#ilginç bir hata yüzünden
from sklearn.preprocessing import LabelEncoder
y_train = keras.utils.to_categorical(LabelEncoder().fit_transform(y_labels))

# y_train = keras.utils.to_categorical(sk.preprocessing.LabelEncoder().fit_transform(y_labels))

In [12]:
#burada yukarıdaki hücrede x_train değişkeninde tutulan giriş verilerini görsel olarak görebiliyoruz
#ilk iki eleman kullanıcı, film "id"leri olmak üzere geri kalan değerler ise ilgili filmin
#sahip olduğu(bu durumda 1 değerini alan) ve/veya olmadığı(bu durumda 0 değerini alan) değerleri tutan türleri görüyoruz
x_train[:3]

array([[3603, 2879,    0,    0,    1,    0,    1,    0,    0,    0,    1,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [ 320, 3751,    1,    1,    1,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [ 424, 1762,    0,    0,    0,    0,    0,    0,    0,    0,    1,
           0,    0,    1,    1,    0,    0,    0,    0,    0,    0]])

In [13]:
#bu hücrede yukarıda anlatılmaya çalışılan mantık görsel olarak görülüyor
#bu dizaynın uygulanma nedeni ise oluşturulan modelin ilgili reel sayı aralığında tahmin yapması yerine
#ilgili puan kategorilerinden(0.5, 2.0, 4.5 gibi) hangisi olduğunu tahmin etmesi yeterli hale gelecek ve
#daha doğru bir çıktı elde edilecek olmasıdır
y_train[:3]

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [14]:
ratings["rating"].unique()

array([5, 4, 3, 2, 1])

In [15]:
del ratings
del movies
# del tags
del y_labels

import gc
gc.collect()

63

In [16]:
#TODO sınıf farklı veri setlerine göre kendini uyumlu hale getiremiyor, daha dinamik bir mimari uygulanması gerekli

class BondNet(keras.Model):
    def __init__(self, features: np.array, labels: np.array, num_unique_users, num_unique_movies, min_rating, max_rating, 
                 rating_step=0.5, embedding_size=32, batch_size=32, **kwds):
        '''bu fonksiyon katmanların ve diğer parameterlerin oluşturulduğu fonksiyondur'''

        super(BondNet, self).__init__(**kwds)

        ##TODO veriyi ayırma statik olarak belirlenmemeli, train fonksiyonunda parametre olarak alınmalı 

        self.data_split = int(0.8 * len(features))

        self.features = features
        self.labels = labels
        #kullanıcılar ve filmler için kullanılacak embedding vektörlerinin uzunluğunun ne kadar olacağının tutulduğu değişken
        self.id_input_size = 2
        self.mfeature_input_size = len(self.features[0, 2:])
        self.output_size = int(((max_rating - min_rating) / rating_step) + 1)
        self.embedding_size = embedding_size
        self.batch_size = batch_size

        #EMBEDDING API
        
        #burada hashing yaparak ilgili kullanıcı ve filmin aşağıda görülen embedding katmanında
        #hangi satıra denk geldiğini haritalandı
        #örneğin "42" numaralı kullanıcının embedding vektörü embedding katmanında "35" numaralı indise(satıra) denk gelebilir 
        #çünkü kullanıcı veya film "id"leri 0'dan başlayarak birer birer artmak zorunda değil veya
        #integer bir değer olmak zorunda bile değil
        #bu yüzden bu değerleri ilgili kullanıcı veya film embedding vektörüne denk gelen satırın indisine göre haritalamamız gerekli
        self.user_table = tf.lookup.StaticHashTable(
            tf.lookup.KeyValueTensorInitializer(np.unique(features[:, 0]), np.arange(num_unique_users)),
            default_value=-1
        )
        self.movie_table = tf.lookup.StaticHashTable(
            tf.lookup.KeyValueTensorInitializer(np.unique(features[:, 1]), np.arange(num_unique_movies)),
            default_value=-1
        )

        #bu embedding vektörleri için alınan girişi barındıran katman
        #yani bu katman ile sadece "userId" ve "movieId" değerlerini alıyoruz
        #çünkü sadece bunlar embedding katmanları için kullanılacak, film türleri için farklı bir API kullanacağız
        self.embedding_input = keras.layers.InputLayer(
            input_shape=(self.id_input_size,),
            dtype=np.int64
        )

        #burada collaborative filtering modelinde kullanılan matrix factorization metodunu kullanmak adına
        #her bir "tekil" kullanıcı ve filme "embedding_size" boyutunda bir embedding vektörü ve 
        #her bir kullanıcı ve film özünde tekil kavramlar olduğu için birer integer sapma değeri(bias) tahsis edildi böylece
        #vektörler üzerinde dot product işlemi kullanılarak ve daha sonra sapma değerleri ile toplanarak ilgili puan elde edilebilir 
        #daha sonra da modelin çıktısında tahmin edilen kullanıcının ilgili filme verdiği puan kategorisini
        #gerçekte olan kategori ile karşılaştırarak ilgili embedding vektörlerini 
        #daha doğru sonucu verdiği değerlere doğru değiştirebiliriz
        #yani ilgili loss değerine göre ilgili vektör ağırlıklarını(weights)
        #daha doğru sonucu verdiği yönde(gradient descent işlemi) değiştirilebilir
        self.user_embedding = keras.layers.Embedding(
            input_dim=num_unique_users,
            output_dim=embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.L1(l1=3e-3)
        )
        self.user_bias = keras.layers.Embedding(
            input_dim=num_unique_users,
            output_dim=1
        )

        self.movie_embedding = keras.layers.Embedding(
            input_dim=num_unique_movies,
            output_dim=embedding_size,
            embeddings_initializer="he_normal",  
            embeddings_regularizer=keras.regularizers.L1(l1=3e-3)
        )
        self.movie_bias = keras.layers.Embedding(
            input_dim=num_unique_movies,
            output_dim=1
        )

        #VISIBLE FEATURES API

        #bu katman ise film türlerini giriş olarak alan katman 
        #burada giriş olarak verilen film türleri ilgili gizli katmanlardan geçtikten sonra(şuan için 3 tane dense layer var)
        #iki boyuttan tek boyuta indirildiği(flatten layer ile) ve daha sonra kullanmak için hazır hale getirildiği katman
        #bu API'ın görevi "call" fonksiyonunda anlatılmıştır
        self.visible_input = keras.layers.InputLayer(
            input_shape=(self.mfeature_input_size,),
            dtype=np.int8
        )
        self.dense1 = keras.layers.Dense(
            self.embedding_size * 16,
            activation="relu"
        )
        self.dense2 = keras.layers.Dense(
            self.embedding_size * 8,
            activation="relu"
        )
        self.dense3 = keras.layers.Dense(
            self.embedding_size * 4,
            activation="relu"
        )
        self.vis_drop = keras.layers.Dropout(0.2)
        self.visible_flatten = keras.layers.Flatten()

        #en önemli kısımlardan biri olan modelin çıkışını aldığımız katmanda ise kullanıcının ilgili filme verdiği puanın
        #hangi kategoriye ait olduğunun tahmini yapılıyor
        #örneğin en düşük puanın(min_rating) 0.0, en yüksek puanın(max_rating) 5.0 ve puan artım oranın(rating_step) 1.0 olduğu senaryoda
        #((5.0 - 0.0) / 1.0) + 1) = 6 olmak üzere toplam 6 tane kategori barındırdığı(bu kategoriler 0.0, 1.0, 2.0, 3.0, 4.0, 5.0)
        #ve çıktının da bu kategorilerden biri olarak seçildiği katmanın olduğu kısım bu çıktı katmanıdır
        #bu örneğe itafen eğer çıktı(çıktılar 6 elemanlı diziler olacak çünkü toplamda 6 kategori var)
        #[0, 0, 1, 0, 0, 0] ise seçilen kategori, kullanıcının ilgili filme "2.0" puanını verdiğini 
        #tahmin eden değeri temsil eden kategoridir
        #[0, 0, 0, 0, 0, 1] ise seçilen kategori, max_rating değeri yani "5.0"dır
        self.output_labels = keras.layers.Dense(
            self.output_size,
            activation="softmax"
        )

    def call(self, inputs):
        '''bu fonksiyon girişlerin modelin ilgili katmanlarına iletilmesini sağlayan fonksiyondur'''
        '''bu fonksiyonda bulunan "inputs" parametresi x_train değişkenin satırlarını barındırır'''
        '''yani kullanıcı, film "id"leri ve filmlerin türleri'''

        #x_train değişkeninde yer alan ilk iki eleman sırası ile "userId" ve "movieId" olduğu için bu "id"lere denk gelen
        #embedding vektörlerini bulmak adına hash tablosunda ilgili satırların indisi bulunur 
        user_index = self.user_table.lookup(inputs[0][:, 0])
        movie_index = self.movie_table.lookup(inputs[0][:, 1])

        #daha sonra ilgili kullanıcılara ve filmlere denk gelen embedding vektörleri ve sapma değerleri(bias)
        #embedding ve sapma değerleri(bias) katmanından çekilir 
        user_vector = self.user_embedding(user_index)
        user_bias = self.user_bias(user_index)

        movie_vector = self.movie_embedding(movie_index)
        movie_bias = self.movie_bias(movie_index)

        #matrix factorization metodunda kullanılan dot product işlemi 
        #ilgili embedding vektörleri üzerinde uygulanır ve sapma değerleri ile toplanarak ilgili değerler elde edilir
        dot_user_movie = tf.tensordot(user_vector, movie_vector, 2)
        embed_out = dot_user_movie + user_bias + movie_bias

        #hatırlanacağı üzere x_train değişkeninde 2. elemandan sonrası film türleri tutuyordu
        #bu film türleri "__init__" fonksiyonunda belirtilen gizli katmanlara giriş olarak verilerek
        #türlerin kodlandığı(encoding) değerler hesaplanır
        genres_vector = self.vis_drop(self.dense3(self.dense2(self.dense1(self.visible_input(inputs[1])))))
        genres_out = self.visible_flatten(genres_vector)             

        #bu embedding değerleri ile tür değerleri birleştirelerek çıkış katmanına giriş olarak verilmeye hazır hale getirilir
        out = tf.concat([embed_out, genres_out], axis=1)

        #ardından son katman olan sınıflandırmanın yapıldığı katmanda ise ilgili kullanıcıların ilgili filmlere 
        #hangi puanı verdiğinin kategorisinin tahmininin yapılması sağlanır
        #daha sonra da, bahseldiği üzere tahmin edilen değer kategorisi ile gerçek değerin kategorisi karşılaştıralarak 
        #ilgili embedding vektörlerinin değerlerinin ve 
        #ilgili filmlerin türlerinin kodlarının(encoding) elde edilmesinin sağlayan
        #rgizli katmanlara(dense layers) matematiksel olarak gömülen değerlerin
        #gerçek değeri elde etmeye doğru iyileştirilmesi sağlanır
        return self.output_labels(out)

    def train_gen(self):
        assert len(self.features) == len(self.labels)

        for i in range(0, self.data_split, self.batch_size):

            curr_batch = self.batch_size
            if (i + self.batch_size) > self.data_split:
                curr_batch = self.data_split - i

            xids = []
            xmovie_features = []
            y = []
            for j in range(curr_batch):
                xids.append(self.features[i + j, 0:self.id_input_size])
                xmovie_features.append(self.features[i + j, self.id_input_size:self.id_input_size + self.mfeature_input_size])
                y.append(self.labels[i + j])

            yield (np.asarray(xids, dtype=np.int64), np.asarray(xmovie_features, dtype=np.int8)), np.asarray(y, dtype=np.int8)

    def val_gen(self):
        assert len(self.features) == len(self.labels)

        for i in range(self.data_split, len(self.features), self.batch_size):

            curr_batch = self.batch_size
            if (i + self.batch_size) > len(self.features):
                curr_batch = len(self.features) - i

            xids = []
            xmovie_features = []
            y = []
            for j in range(curr_batch):
                xids.append(self.features[i + j, 0:self.id_input_size])
                xmovie_features.append(self.features[i + j, self.id_input_size:self.id_input_size + self.mfeature_input_size])
                y.append(self.labels[i + j])

            yield (np.asarray(xids, dtype=np.int64), np.asarray(xmovie_features, dtype=np.int8)), np.asarray(y, dtype=np.int8)

    #alttaki iki fonksiyon modele girişlerin uygun formata getirilmesini sağlar
    def get_train_dataset(self):

        #veri belirlenen ayırma değerine göre eğitim ve doğrulama verisi olarak iki farklı veri setine ayrılır
        #bu veri setleri gruplanarak ve karıştıralarak modelin veriyi ezberlemesi daha zor hale getirilir
        # x_train, y_train = (
        #     self.features[:self.data_split],
        #     self.labels[:self.data_split]
        # )
        
        # train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
        # train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

        train_dataset = tf.data.Dataset.from_generator(
            self.train_gen,
            output_signature=(
                (tf.TensorSpec(shape=(None, self.id_input_size), dtype=tf.int64),
                tf.TensorSpec(shape=(None, self.mfeature_input_size), dtype=tf.int8)),
                tf.TensorSpec(shape=(None, self.output_size), dtype=tf.int8)
            )
        )

        return train_dataset

    def get_validation_dataset(self):
        # x_val, y_val = (
        #     self.features[self.data_split:],
        #     self.labels[self.data_split:]
        # )

        # val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
        # val_dataset = val_dataset.shuffle(buffer_size=1024).batch(64)

        val_dataset = tf.data.Dataset.from_generator(
            self.val_gen,
            output_signature=(
                (tf.TensorSpec(shape=(None, self.id_input_size), dtype=tf.int64),
                tf.TensorSpec(shape=(None, self.mfeature_input_size), dtype=tf.int8)),
                tf.TensorSpec(shape=(None, self.output_size), dtype=tf.int8)
            )
        )

        return val_dataset

    #TODO model fonksiyonlarının kendi implementasyon biçimleri ile çağrılması gerek, keras API'i ile değil

    def cmpl():
        pass
    def trn():
        pass

In [26]:
bondnet = BondNet(
    x_train,
    y_train,
    num_unique_users,
    num_unique_movies,
    min_rating,
    max_rating,
    rating_step=1.0,
    embedding_size=64
)

#modelde kullanılacak loss ve optimizer algoritmaları seçilir
bondnet.compile(
    optimizer="Adam",
    loss="MeanSquaredLogarithmicError",
    metrics=[
        tf.keras.metrics.MeanSquaredError(),
        tf.keras.metrics.RootMeanSquaredError(),
        "accuracy"
    ]
)

In [27]:
td = bondnet.get_train_dataset()
vd = bondnet.get_validation_dataset()

list(td.take(1))

[((<tf.Tensor: shape=(32, 2), dtype=int64, numpy=
   array([[3603, 2879],
          [ 320, 3751],
          [ 424, 1762],
          [4821, 1196],
          [1216, 1963],
          [1820, 2455],
          [2353, 2757],
          [2343, 2946],
          [3889,  319],
          [3749,  708],
          [4131, 3576],
          [3681, 3101],
          [5026,  227],
          [2383, 1235],
          [4781, 2324],
          [1141, 1073],
          [ 753, 1073],
          [1806,  168],
          [4387, 1947],
          [3312, 1347],
          [ 787, 2427],
          [1274,  480],
          [4169, 1873],
          [ 206,  611],
          [4448, 3726],
          [4310, 2146],
          [3927,  516],
          [3935, 3717],
          [ 696, 3702],
          [2056, 1971],
          [1920,  671],
          [4345, 2396]])>,
   <tf.Tensor: shape=(32, 19), dtype=int8, numpy=
   array([[0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [29]:
#modelin eğitim aşaması fonksiyonlardan alınan veri setleri ile başlatılır
history = bondnet.fit(
    td.repeat(),
    epochs=3,
    batch_size=bondnet.batch_size,
    steps_per_epoch=len(bondnet.features[:bondnet.data_split]) // bondnet.batch_size,
    validation_steps=len(bondnet.features[bondnet.data_split:]) // bondnet.batch_size,
    validation_data=vd.repeat()
)
history.history

Epoch 1/3
25005/25005 [==============================] - 652s 26ms/step - loss: 0.3156 - mean_squared_error: 0.1367 - root_mean_squared_error: 0.3698 - accuracy: 0.4269 - val_loss: 0.2930 - val_mean_squared_error: 0.1339 - val_root_mean_squared_error: 0.3660 - val_accuracy: 0.4432
Epoch 2/3
25005/25005 [==============================] - 633s 25ms/step - loss: 0.2929 - mean_squared_error: 0.1335 - root_mean_squared_error: 0.3653 - accuracy: 0.4480 - val_loss: 0.2955 - val_mean_squared_error: 0.1335 - val_root_mean_squared_error: 0.3653 - val_accuracy: 0.4461
Epoch 3/3
25005/25005 [==============================] - 632s 25ms/step - loss: 0.2914 - mean_squared_error: 0.1331 - root_mean_squared_error: 0.3649 - accuracy: 0.4503 - val_loss: 0.2920 - val_mean_squared_error: 0.1333 - val_root_mean_squared_error: 0.3652 - val_accuracy: 0.4474


{'loss': [0.3156047761440277, 0.2929003834724426, 0.2913684844970703],
 'mean_squared_error': [0.13671597838401794,
  0.13346238434314728,
  0.133125901222229],
 'root_mean_squared_error': [0.3697519898414612,
  0.3653261065483093,
  0.36486348509788513],
 'accuracy': [0.42689836025238037, 0.447986900806427, 0.4502902626991272],
 'val_loss': [0.29298484325408936, 0.29545170068740845, 0.2919926941394806],
 'val_mean_squared_error': [0.13394276797771454,
  0.13347351551055908,
  0.13334234058856964],
 'val_root_mean_squared_error': [0.3659816086292267,
  0.36534053087234497,
  0.3651602268218994],
 'val_accuracy': [0.44323909282684326,
  0.4461336135864258,
  0.44742339849472046]}

In [25]:
bondnet.summary()

Model: "bond_net_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 2)]               0         
                                                                 
 embedding_4 (Embedding)     multiple                  386560    
                                                                 
 embedding_5 (Embedding)     multiple                  6040      
                                                                 
 embedding_6 (Embedding)     multiple                  237184    
                                                                 
 embedding_7 (Embedding)     multiple                  3706      
                                                                 
 input_4 (InputLayer)        [(None, 19)]              0         
                                                                 
 dense_4 (Dense)             multiple                  2